In [2]:
import sys
sys.path.append('/mnt/d/Lab/16S-Taxa-Phlyo/scripts')

In [3]:
from functions import *

usage: ipykernel_launcher.py [-h] -i INDIR -o OUTDIR [-d DATABASE] [-r INDEX] [-j JOBS] [-t THREADS]
ipykernel_launcher.py: error: the following arguments are required: -i/--input, -o/--output


SystemExit: 2

/home/junyu/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
Root = "/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S.fasta"

In [5]:
OutDir = "/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S"

In [11]:
import os

In [7]:
from Bio import SeqIO

In [12]:
for seq in SeqIO.parse(Root, "fasta"):
    SeqIO.write(seq, os.path.join(OutDir, seq.id+".fasta"), "fasta")

In [18]:
fastaFileList = []
InDir = "/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S"
for file in os.listdir(InDir):
    filePath = os.path.join(InDir, file)
    if file.endswith(".fasta") and os.path.getsize(filePath) > 0:
        fastaFileList.append(filePath)

In [19]:
fastaFileList

['/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf13.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf176.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf180.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf182.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf187.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf189.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf194.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf196.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf201.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf216.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf359.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf394.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf404.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf405.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf406.fasta',
 '/mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf41.fasta',
 '/mnt/d/L

In [20]:
db = "/mnt/d/Lab/16S-Taxa-Phlyo/database/SILVA_138.1_SSURef_NR99_tax_silva.fasta"

In [21]:
indexPath = "/mnt/d/Lab/16S-Taxa-Phlyo/database/silva-138-99-index.tsv"

In [22]:
OutDir = "/mnt/d/Lab/16S-Taxa-Phlyo/result/All_root_16S"

In [26]:
import os
import argparse
import collections
import pandas as pd
import subprocess
from Bio import SeqIO

from itertools import repeat
from multiprocessing import Pool, freeze_support

In [24]:
def RunBlastnParallel(fastaList, db, jobs, threads, OutDir):
    pool = Pool(processes=jobs)
    pool.starmap(RunBlastn, zip(fastaList, repeat(db), repeat(threads), repeat(OutDir)))
    pool.close()
    pool.join()
    pool.terminate()
def RunBlastn(fasta, db, threads, OutDir):
    blastDir = os.path.join(OutDir, "blast")
    if os.path.exists(blastDir) == 0:
        os.makedirs(blastDir, 0o777, True)
    OutFile = os.path.join(blastDir, os.path.split(fasta)[1].replace(".fasta", "") + "_blast.tsv")
    cmd = "blastn -query " + fasta  + " -out " + OutFile + " -evalue 1.0 -max_target_seqs 5 -outfmt 6 -db " + db + " -num_threads " + str(threads) 
    print(cmd)
    subprocess.call(cmd, shell=True)

def parseIndex(blastDir, OutDir):
    taxaList = []
    df = pd.DataFrame()
    out = pd.DataFrame()
    for file in os.listdir(blastDir):
        #print(file)
        if file.endswith(".tsv") and os.path.getsize(os.path.join(blastDir, file)) > 0:
            df1 = pd.read_table(os.path.join(blastDir, file), header = None)
            df1.columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
            df = df.append(df1[0:1])
    df = df.reset_index()
    df2 = pd.read_table(indexPath)
    index = dict(zip(df2["ID"], df2["Taxa"]))
    for i in range(len(df)):
        taxa = index[df["sseqid"][i]]
        taxaList.append(taxa)
    out["qseqid"] = df["qseqid"]
    out["taxa"] = taxaList
    out["pident"] = df["pident"]
    out["length"] = df["length"]
    return out

In [27]:
## blastn
RunBlastnParallel(fastaFileList, db, 2, 4, OutDir)
## blast tsv parse
pairsOut = parseIndex(os.path.join(OutDir, "blast"), OutDir)
pairsOut.to_csv(os.path.join(OutDir, "pairsOut.csv"))

blastn -query /mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Root1204.fasta -out /mnt/d/Lab/16S-Taxa-Phlyo/result/All_root_16S/blast/Root1204_blast.tsv -evalue 1.0 -max_target_seqs 5 -outfmt 6 -db /mnt/d/Lab/16S-Taxa-Phlyo/database/SILVA_138.1_SSURef_NR99_tax_silva.fasta -num_threads 4blastn -query /mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf13.fasta -out /mnt/d/Lab/16S-Taxa-Phlyo/result/All_root_16S/blast/Leaf13_blast.tsv -evalue 1.0 -max_target_seqs 5 -outfmt 6 -db /mnt/d/Lab/16S-Taxa-Phlyo/database/SILVA_138.1_SSURef_NR99_tax_silva.fasta -num_threads 4

blastn -query /mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Leaf176.fasta -out /mnt/d/Lab/16S-Taxa-Phlyo/result/All_root_16S/blast/Leaf176_blast.tsv -evalue 1.0 -max_target_seqs 5 -outfmt 6 -db /mnt/d/Lab/16S-Taxa-Phlyo/database/SILVA_138.1_SSURef_NR99_tax_silva.fasta -num_threads 4
blastn -query /mnt/d/Lab/16S-Taxa-Phlyo/data/All_root_16S/Root1212.fasta -out /mnt/d/Lab/16S-Taxa-Phlyo/result/All_root_16S/blast/Root1212_blast.tsv -eval